# Export routine for `bw2extdb`

This notebook explains how the exporting routine works for a toy data example and a local SQLite database.

In [ ]:
import bw2extdb.exportImport.exporter as exporter
import bw2extdb.exportImport.database as database

import datetime
import sqlite3

import bw2io
import bw2data

## Download the toy example and set up Brightway project
Here the brightway project from which the dataset is exported from.

In [ ]:
project_name = 'bw2extdb_test_export'
bw2data.projects.set_current(project_name)
bw2io.bw2setup()
bw2io.data.add_example_database(overwrite=False)

## Define database for export
The exporter can export one or many databases from the project at once. When multiple are selected they will be saved as one dataset in the SQL database, which means when that dataset is imported it will be imported as one database into the Brightway project. The reason is that in the project to be exported from the data might be split up into multiple Brightway databases for readability or structure, though they all belong together, e.g., imagine an LCA on a battery, there might be the BW databases "Anode", "Cathode", "Assembly", and "End-of-life" though they all describe the "battery" database, in this case you would specify `databases` as `["Anode", "Cathode", "Assembly", "End-of-life"]`. They would all be merged into one "battery" dataset on the SQL server.

Let's have a look what databases there are:

In [ ]:
bw2data.databases

This toy example only has one database we want to export, so:

In [ ]:
databases = ['Mobility example']

### Bisphere version

Whenever you are working with the EcoInvent biosphere databases, which is the standard one when seting up a brightway project, you need to know with which biosphere database version you are working with. Since if you try to import the dataset later and are using another `biosphere3` database in that Brightway project, some Biosphere flows might not be able to match. There are two relevant versions `3.8` and `3.9`. The biosphere version of your `biosphere3` database is set when you run `bw2setup()` and is dependent on the `bw2io` version of the virtual environment you are in. Read all about it here: https://github.com/brightway-lca/brightway2-io 

This toy example has no ecoinvent biosphere database, so we do not need to specify a `bioshere_verion`. Or just say it is `None`:

In [ ]:
biosphere_version = None

## Create SQL connection
The SQL connection is managed by the `engine`. It is a `sqlalchemy` object which is the "home base" for the actual database and should only exist once for every connection https://docs.sqlalchemy.org/en/20/core/engines.html

We are using SQLite to demonstrate the import routine. Alternative the engine can be created for any other type of SQL database supported by `sqlalchemy` (https://docs.sqlalchemy.org/en/20/core/engines.html#supported-databases), e.g., PostgreSQL, MySQL, MicrosoftSQL. Some of the SQL database engine creation are wrapped as methods in the `bw2extdb.exportImport.database` module, e.g. `create_sqlite_engine` or `create_MSsql_engine`.

Set the absolute file path of where you want the sqlite database file to be stored on your disk in `sqlite_file_path_abs`.

In [ ]:
sqlite_file_path_abs: str = '/Users/.../bw2extdb/examples/example_database.db' # Change this!

We can just ise the `create_sqlite_engine` method in the `database` module of `bw2extdb`. When the egnine is created we must also create the database and the tables in the SQL database, using `create_db_and_tables`. When the database already exist this method only checks if the database model we have matches the database model in the SQL database.

In [ ]:
engine = database.create_sqlite_engine(sqlite_file_path_abs)
database.create_db_and_tables(engine)

## Initialize the LCIExporter 

The `LCIExporter` is based on the CSV export in `bw2io.export.csv` (https://github.com/brightway-lca/brightway2-io/blob/main/bw2io/export/csv.py) and the exporting routine follows the standard Brightway exporting routine. In general this class extracts the data into object relational mappers (ORM) and are the objects mapping the extracted data to the SQL database. 

### For the SQL enthusiasts
This implementation is using `sqlmodel` package which is a wrapping of `pydantic` and `sqlalchemy` and allows us to check the data if it upholds a specified format given by the classes in `exportImport.models` module. And then directly send it to any SQL database with this structure. Read more about it here https://sqlmodel.tiangolo.com/, 

In [ ]:
LCIExporter = exporter.LCIExporter(
    project_name=project_name, 
    databases=databases, 
    engine=engine, 
    biosphere_version=biosphere_version
    )

### Extract the LCI data

The activities from the specified databases in extracted. We extract the emission activities and the process activities into two different sets, since they have different data structure.

Since we are exporting to a specific data structure all variables in the acitvities and exchanges which are not in the SQL datastructure will not be exorted, every variable which is not exported is returned as a warning below.

In [ ]:
processactivities, emissionactivities = LCIExporter.extract_lci_data()

### Define the metadata

The dataset we are exporting to the SQL database must have a descriptive metadata, so that we at a later point can understand what this dataset entails and when it was created and so on.

The current metdata to be specified is:

`dataset_name`: The name of the dataset, e.g., 'Battery'

`dataset_final_date`: The date when the dataset was finialized or the project was finalized

`description`: A detailed description of the dataset including all assumption and literature

`user_email_addres`: The email address of the user which exports the data

`keywords_input`: A list of keywords describing the dataset

In [ ]:
dataset_name = 'Mobility example'
dataset_final_date = datetime.date(2022,5,11)
description = 'This is a Mobility database with an electic and fuel car and is used as an exmple to demonstrate the export routine for bw2extdb'
user_email_addres = 'test.user@protonmail.com'
keywords_input = ['test', 'Mobility', 'example']

In [ ]:
datasetmetadata = LCIExporter.create_metadata(
    dataset_name=dataset_name, 
    dataset_final_date=dataset_final_date, 
    description=description,
    user_email_addres=user_email_addres,
    keywords_input=keywords_input
    )

### Check completeness of exported LCI data

This method will run through the complete export and import routine using an in memory SQLite database and a temporary Brightway project to see if the exported data can be fully recreated to original data. This method does not need to be run.

In [ ]:
LCIExporter.check_activities_completeness(
    processactivities, datasetmetadata, emissionactivities=emissionactivities
    )


## Export the data to the SQL database

This is the final step and the specified metadata and the extracted LCI data is exported to the SQL database

In [ ]:
LCIExporter.export_to_sql(processactivities, datasetmetadata, emissionactivities=emissionactivities)

Let us check if the data has been exported using the sqlite package. This ofcourse only works if we have exported it to a sql file.

In [ ]:
conn = sqlite3.connect(sqlite_file_path_abs)
c = conn.cursor()
c.execute(
    "SELECT * FROM datasetmetadata WHERE dataset_name = ?", (dataset_name,)
)
result = c.fetchall()
conn.close()
result